# COGS 118B - Final Project

# Finding Waldo 

## Group members

- Lillian Wood
- Taha Alam
- Zichen ‘Cardiff’ Jiang
- Will Lutz
- Sara Shao

# Introduction
In our project, we will perform PCA analysis on images of the character, Waldo, from Where’s Waldo to identify whether or not a test image is a picture of Waldo.

Utilizing a dataset of black and white images of Waldo, we will reduce the dimensionality of the vectorized images by calculating the eigenvalues and eigenvectors of the original image to reconstruct an image of Waldo in a smaller subspace. Once the eigenface is constructed using x number of principal components, we will be able to calculate the Euclidean distance between this constructed image and test images of both Waldo and non-Waldo projected into the same subspace to identify if the image is a picture of Waldo. The goal of this project is to elaborate on the topic of PCA that was discussed in class and present it in a fun and interactive way.

This project is important because finding Waldo manually is extremely tricky and time-consuming, sometimes ranging. Therefore, this intuitive PCA method to automatically find Waldo is needed.

# Related Work

Our project is based on the PCA example presented in class and in the homework. We will take the same steps to create the eigenfaces of our training and test sets of images. Once we create the eigenfaces for Waldo, we will follow the methods described in Liton Chandra Paul and Abdulla Al Sumam (2012) <a name="relatedwork"></a> [1](#relatedwork) in order to calculate the distance between the eigenfaces of the test images and the eigenfaces of the training images when projected into the subspace.

# Methods
Briefly, we will collect 31 Waldo images, randomly vectorize 30 of them, and put these 30 vectors into a matrix. Similarly this will be done for 1 non-Waldo image. The Waldo image matrix will be used for PCA principle component generation. The principle components will be used to recontruct existing and one new Waldo image and one non-Waldo image. The quality of the reconstruction will be measured using reconstruction distance calculation.

### Vectorizing Images
Images are 

### PCA
Explaination

### Reconstruction Distance Function
Explaination

# Results
Results - What did you discover? How well did it work?  As this is a class project, it is likely that many things did not work as well as planned.  For this project, detailing what went wrong is as important as describing what went well.  (approx 7 points)

### Preparing the images

vectorizing images explanation and code below

In [1]:
import numpy as np
import scipy.io as sio
import matplotlib
import matplotlib.pyplot as plt
from numpy.matlib import repmat
from sklearn.preprocessing import normalize
import time
from PIL import Image
import random
%matplotlib inline
import os

In [2]:
# Ex. waldo_matrix = vectorize(262144, 31, 'waldo')
# For the 31 Waldo pictures, each is turned into a 262144 by 1 (256 x 256 x 4) column vector. So each col of this 262144 by 31 matrix is one of the pictures.
def vectorize(pic_num, pic_dim, pic_dir):
    A = np.empty([pic_num, pic_dim])

    file_ls = []
    for filename in os.listdir(pic_dir):
        f = os.path.join(pic_dir, filename)
        file_ls.append(f)

    new_image_i = random.randrange(30)
    for i in range(len(file_ls)):
        if i == new_image_i:
            continue
        else:
            f = file_ls[i]
            img = Image.open(f).convert('RGBA')
            arr = np.array(img)
            flat_arr = arr.ravel()
            v = np.matrix(flat_arr)
            col_v = v.T
            A[:, i-1:i] = col_v
            i += 1
    return A

In [3]:
# Ex. viewimage(waldo_input[:, 8])
def viewimage(vector):
    # Showing the 8th picture from the matrix
    vector_uint8 = vector.astype(np.uint8)
    shape = (256, 256, 4)
    reconstruct_arr = np.asarray(vector_uint8).reshape(shape)
    reconstruct_img = Image.fromarray(reconstruct_arr, 'RGBA')
    reconstruct_img.show()

In [4]:
waldo_matrix = vectorize(262144, 30, 'waldo')
waldo_matrix.shape

(262144, 30)

In [5]:
viewimage(waldo_matrix[:, 18])

### PCA

PCA explanation and code below

### Reconstruction Distance Function

 explanation and code below

# Comparison to Related Work?

if we have time, write about existing github repo

# Discussion
What did you learn?  What could you do better? (What would you
have done next if you had more time)?.....  Why do you think it didn't work if it didn't?  
If everything worked perfectly,  what next steps would you suggest for follow-up work.  For full credit discuss two extensions or improvements to your project with short justifications for why you think that would work better (improvements) or why they are promising extensions. (approx 7 points) 

# Author Contribution
| Task      | Assignee |
| ----------- | ----------- |
| Introduction writing     | Will       |
| Related Work writing  | Will        |
| Vectorizing Images and methods writing   | Cardiff        |
| PCA and methods writing      | Sara       |
| Reconstruction Distance Function and methods writing   | Lilly and Taha        |
| Comparison to Related Work writing      |    []    |
| Discussion writing  |  all     |
| Editing | all |

# Footnotes

<a name="relatedwork"></a>1.[^](#relatedwork): last name, first name. (year). name of article. article link<br>